In [1]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session

In [2]:
session = get_session()

### Deploy the model from stage location as a UDF

In [44]:
session.clear_imports()
session.clear_packages()
from snowflake.snowpark.functions import udf,call_udf

# Add trained model as dependency
session.add_import("@models/Predictive_Maintenance_model_20230919_194031.joblib.gz")
import pandas
@udf(name='predict', session=session,is_permanent=True,replace=True,stage_location="@SCORE",packages=["snowflake-snowpark-python","pandas", "joblib","scikit-learn"])
def score(payload: list) -> str:
    import sys
    import pandas as pd
    from joblib import load

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]

    pipeline_file = import_dir + 'Predictive_Maintenance_model_20230919_194031.joblib.gz'
    pipeline = load(pipeline_file)

    prediction = pipeline.predict(df)[0]
    
    return prediction

The version of package joblib in the local environment is 1.3.2, which does not fit the criteria for the requirement joblib. Your UDF might not work when the package version is different between the server and your local environment


#### Sample Input 

In [62]:
inp =[6,407438,1,3,'S1F0',1,0,26953834,0,0,2.079441541679836,0.0,3.9702919135521215,0.0,2.079441541679836]

Consuming the model as a 

In [59]:
output = session.sql("select PREDICT("+str(inp)+")").collect()

In [60]:
output

[Row(PREDICT([6, 407438, 1, 3, 'S1F0', 1, 0, 26953834, 0, 0, 2.079441541679836, 0.0, 3.9702919135521215, 0.0, 2.079441541679836])='0')]